# Random Forest : Prosper Loan Dataset

A decision tree a learned set of rules that allows us to make decisions on data.

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

In [1]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

Initializing Spark...
Spark found in :  /home/ubuntu/spark
Spark config:
	 spark.app.name=TestApp
	spark.master=local[*]
	executor.memory=2g
	spark.sql.warehouse.dir=/tmp/tmpsdh5f838
	some_property=some_value
Spark UI running on port 4042


## Step 1: Load the Data

In [2]:
## small file, start with this
datafile = "/data/prosper-loan/prosper-loan-data-sample.csv"

## this is a large file
#datafile = "/data/prosper-loan/prosper-loan-data.csv.gz"

In [3]:
%%time

data = spark.read. \
          option("header", "true"). \
          option("inferSchema", "true").  \
          csv(datafile)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.89 s


In [4]:
print("read {:,} records".format(data.count()))
# schema
data.printSchema()

read 200 records
root
 |-- Term: integer (nullable = true)
 |-- LoanStatus: integer (nullable = true)
 |-- BorrowerRate: double (nullable = true)
 |-- ProsperRating (numeric): double (nullable = true)
 |-- ProsperScore: double (nullable = true)
 |-- ListingCategory: string (nullable = true)
 |-- BorrowerState: string (nullable = true)
 |-- EmploymentStatus: string (nullable = true)
 |-- EmploymentStatusDuration: double (nullable = true)
 |-- IsBorrowerHomeowner: boolean (nullable = true)
 |-- CreditScore: double (nullable = true)
 |-- CurrentCreditLines: double (nullable = true)
 |-- OpenCreditLines: double (nullable = true)
 |-- TotalCreditLinespast7years: double (nullable = true)
 |-- OpenRevolvingAccounts: integer (nullable = true)
 |-- OpenRevolvingMonthlyPayment: double (nullable = true)
 |-- InquiriesLast6Months: double (nullable = true)
 |-- TotalInquiries: double (nullable = true)
 |-- CurrentDelinquencies: double (nullable = true)
 |-- AmountDelinquent: double (nullable = true

In [5]:
## print with pandas
data.limit(10).toPandas()

,Term,LoanStatus,BorrowerRate,ProsperRating (numeric),ProsperScore,ListingCategory,BorrowerState,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,...,ProsperPaymentsOneMonthPlusLate,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,LoanOriginalAmount,MonthlyLoanPayment,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,YearsWithCredit
0,36,1,0.0789,7.0,9.0,Debt,VA,Employed,9.0,True,...,0.0,0.0,0.00,17300,541.24,0,0,0.0,169,20
1,36,0,0.1450,4.0,6.0,Unknown,OR,Part-time,15.0,False,...,0.0,0.0,0.00,9500,327.00,0,0,0.0,171,23
2,36,1,0.1707,4.0,6.0,Debt,UT,Full-time,22.0,True,...,0.0,0.0,0.00,7200,256.95,0,0,0.0,216,16
3,36,1,0.0830,7.0,9.0,Other,VA,Full-time,175.0,True,...,0.0,0.0,0.00,4500,141.64,0,0,0.0,151,18
4,36,1,0.1550,4.0,6.0,Debt,IL,Full-time,18.0,False,...,0.0,0.0,0.00,8000,277.14,0,0,0.0,140,14
5,36,1,0.2099,4.0,6.0,Other,IN,Employed,77.0,True,...,0.0,0.0,0.00,4000,150.68,0,0,0.0,74,22
6,36,1,0.0950,7.0,9.0,Auto,MI,Full-time,43.0,True,...,0.0,10000.0,5584.95,6000,192.20,0,0,0.0,216,21
7,36,1,0.0670,4.0,6.0,Business,PA,Full-time,40.0,False,...,0.0,0.0,0.00,1250,38.43,0,0,0.0,70,14
8,36,1,0.1330,4.0,6.0,Unknown,MD,Self-employed,5.0,True,...,0.0,0.0,0.00,8000,270.71,0,0,0.0,242,24
9,36,1,0.1000,4.0,6.0,Personal,NY,Full-time,42.0,True,...,0.0,0.0,0.00,25000,797.25,0,0,0.0,730,30


In [6]:
## select a few columns 
## start with: 'LoanStatus',  'EmploymentStatus', 'CreditScore', 'StatedMonthlyIncome'
## we add more later

select_columns = ['LoanStatus',  'ProsperScore', 'EmploymentStatus', 'CreditScore', 'StatedMonthlyIncome', 'ListingCategory']

## Note : vector columns can only have Numbers, don't include Categorical columns here
## And dfefinitely not 'LoanStatus'  (if you are curiuos include and see what happens!)
vector_columns = [  'ProsperScore', 'EmpIndex', 'CreditScore', 'StatedMonthlyIncome']



In [7]:
## Display

prosper = data.select(select_columns)  
prosper.printSchema()

prosper.limit(10).toPandas()

root
 |-- LoanStatus: integer (nullable = true)
 |-- ProsperScore: double (nullable = true)
 |-- EmploymentStatus: string (nullable = true)
 |-- CreditScore: double (nullable = true)
 |-- StatedMonthlyIncome: double (nullable = true)
 |-- ListingCategory: string (nullable = true)



,LoanStatus,ProsperScore,EmploymentStatus,CreditScore,StatedMonthlyIncome,ListingCategory
0,1,9.0,Employed,740.0,10416.666667,Debt
1,0,6.0,Part-time,760.0,0.000000,Unknown
2,1,6.0,Full-time,680.0,5000.000000,Debt
3,1,9.0,Full-time,800.0,5000.000000,Other
4,1,6.0,Full-time,600.0,2625.000000,Debt
5,1,6.0,Employed,760.0,4992.666667,Other
6,1,9.0,Full-time,820.0,4583.333333,Auto
7,1,6.0,Full-time,780.0,2216.666667,Business
8,1,6.0,Self-employed,680.0,8433.333333,Unknown
9,1,6.0,Full-time,820.0,16666.666667,Personal


## Step 2 : Clean Data

In [8]:
# Drop any NA values.  Using `dataframe.na.drop()`
prosper_clean = prosper.na.drop()
print("Original record count {:,}, cleaned records count {:,},  dropped {:,}"\
      .format(prosper.count(), prosper_clean.count(), (prosper.count() - prosper_clean.count())))
prosper_clean.show()


Original record count 200, cleaned records count 200,  dropped 0
+----------+------------+----------------+-----------+-------------------+---------------+
|LoanStatus|ProsperScore|EmploymentStatus|CreditScore|StatedMonthlyIncome|ListingCategory|
+----------+------------+----------------+-----------+-------------------+---------------+
|         1|         9.0|        Employed|      740.0|       10416.666667|           Debt|
|         0|         6.0|       Part-time|      760.0|                0.0|        Unknown|
|         1|         6.0|       Full-time|      680.0|             5000.0|           Debt|
|         1|         9.0|       Full-time|      800.0|             5000.0|          Other|
|         1|         6.0|       Full-time|      600.0|             2625.0|           Debt|
|         1|         6.0|        Employed|      760.0|        4992.666667|          Other|
|         1|         9.0|       Full-time|      820.0|        4583.333333|           Auto|
|         1|         6.0|

### Look at some summary data

In [9]:
prosper_clean.groupBy('LoanStatus').count().show()
prosper_clean.groupBy('EmploymentStatus').count().show()

+----------+-----+
|LoanStatus|count|
+----------+-----+
|         1|  136|
|         0|   64|
+----------+-----+

+----------------+-----+
|EmploymentStatus|count|
+----------------+-----+
|        Employed|   73|
|       Part-time|    6|
|   Self-employed|   11|
|    Not employed|    1|
|           Other|    1|
|       Full-time|  106|
|         Retired|    2|
+----------------+-----+



**=> What does that say about the cardinality of these categorical columns? ***



## Step 3: Converting Categorical columns 

Convert categorical columns to numeric.   
Here let's convert **EmploymentStatus** column

In [10]:
from pyspark.ml.feature import StringIndexer

strIndexer_employment = StringIndexer(inputCol="EmploymentStatus", outputCol="EmpIndex")
prosper_indexed = strIndexer_employment.fit(prosper_clean).transform(prosper_clean)

prosper_indexed.limit(10).toPandas()


,LoanStatus,ProsperScore,EmploymentStatus,CreditScore,StatedMonthlyIncome,ListingCategory,EmpIndex
0,1,9.0,Employed,740.0,10416.666667,Debt,1.0
1,0,6.0,Part-time,760.0,0.000000,Unknown,3.0
2,1,6.0,Full-time,680.0,5000.000000,Debt,0.0
3,1,9.0,Full-time,800.0,5000.000000,Other,0.0
4,1,6.0,Full-time,600.0,2625.000000,Debt,0.0
5,1,6.0,Employed,760.0,4992.666667,Other,1.0
6,1,9.0,Full-time,820.0,4583.333333,Auto,0.0
7,1,6.0,Full-time,780.0,2216.666667,Business,0.0
8,1,6.0,Self-employed,680.0,8433.333333,Unknown,2.0
9,1,6.0,Full-time,820.0,16666.666667,Personal,0.0


## Step 4: Build feature vectors using VectorAssembler.

In [11]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=vector_columns, outputCol="features")
feature_vector = assembler.transform(prosper_indexed)
feature_vector = feature_vector.withColumn("label", feature_vector["LoanStatus"])

feature_vector.limit(10).toPandas()

,LoanStatus,ProsperScore,EmploymentStatus,CreditScore,StatedMonthlyIncome,ListingCategory,EmpIndex,features,label
0,1,9.0,Employed,740.0,10416.666667,Debt,1.0,"[9.0, 1.0, 740.0, 10416.666667]",1
1,0,6.0,Part-time,760.0,0.000000,Unknown,3.0,"[6.0, 3.0, 760.0, 0.0]",0
2,1,6.0,Full-time,680.0,5000.000000,Debt,0.0,"[6.0, 0.0, 680.0, 5000.0]",1
3,1,9.0,Full-time,800.0,5000.000000,Other,0.0,"[9.0, 0.0, 800.0, 5000.0]",1
4,1,6.0,Full-time,600.0,2625.000000,Debt,0.0,"[6.0, 0.0, 600.0, 2625.0]",1
5,1,6.0,Employed,760.0,4992.666667,Other,1.0,"[6.0, 1.0, 760.0, 4992.666667]",1
6,1,9.0,Full-time,820.0,4583.333333,Auto,0.0,"[9.0, 0.0, 820.0, 4583.333333]",1
7,1,6.0,Full-time,780.0,2216.666667,Business,0.0,"[6.0, 0.0, 780.0, 2216.666667]",1
8,1,6.0,Self-employed,680.0,8433.333333,Unknown,2.0,"[6.0, 2.0, 680.0, 8433.333333]",1
9,1,6.0,Full-time,820.0,16666.666667,Personal,0.0,"[6.0, 0.0, 820.0, 16666.666666999998]",1


## Step 5: Split Data into training and test.

We will split our the data up into training and test.  (You know the drill by now).

**=> TODO: Split dataset into 70% training, 30% validation**


In [12]:
# Split the data into training and test sets (30% held out for testing)
(training, test) =  feature_vector.randomSplit([.7,.3])
print("training set = " , training.count())
print("testing set = " , test.count())

training set =  147
testing set =  53


## Step 6: Random Forest

### 6.1 Create RF

In [15]:
from pyspark.ml.classification import RandomForestClassifier

## TODO : Create a RandomForest with numTrees=20  and maxBins=10000

rf = RandomForestClassifier(labelCol="label", featuresCol="features", \
                            numTrees=20, maxBins=10000)

### 6.2 Train the RF

In [16]:
%%time
print ("training starting...")
## TODO : train in training data
rf_model = rf.fit(training)
print ("training done.")

## TODO : NOtice the time it took for training
## Is it more or less than decision trees?

training starting...
training done.
CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 1.69 s


### 6.3 Print RF
**==>Q : How many nodes the tree has?**

In [17]:
## print the model
print(rf_model)
print()
print(rf_model.toDebugString)

RandomForestClassificationModel (uid=RandomForestClassifier_108c5f683403) with 20 trees

RandomForestClassificationModel (uid=RandomForestClassifier_108c5f683403) with 20 trees
  Tree 0 (weight 1.0):
    If (feature 3 <= 7166.666666499999)
     If (feature 2 <= 550.0)
      Predict: 0.0
     Else (feature 2 > 550.0)
      If (feature 3 <= 7014.791666499999)
       If (feature 1 in {3.0})
        Predict: 0.0
       Else (feature 1 not in {3.0})
        Predict: 1.0
      Else (feature 3 > 7014.791666499999)
       Predict: 0.0
    Else (feature 3 > 7166.666666499999)
     If (feature 3 <= 12083.333333)
      If (feature 2 <= 690.0)
       If (feature 3 <= 8316.666666500001)
        Predict: 1.0
       Else (feature 3 > 8316.666666500001)
        If (feature 3 <= 8583.333333499999)
         Predict: 0.0
        Else (feature 3 > 8583.333333499999)
         Predict: 1.0
      Else (feature 2 > 690.0)
       Predict: 1.0
     Else (feature 3 > 12083.333333)
      Predict: 0.0
  Tree 1 (we

### 6.4 Create Predictions

In [18]:
## TODO : predict on test data
predictions = rf_model.transform(test)

predictions2= predictions.drop('rawPrediction', 'probability')
predictions2.show()


+----------+------------+----------------+-----------+-------------------+---------------+--------+--------------------+-----+----------+
|LoanStatus|ProsperScore|EmploymentStatus|CreditScore|StatedMonthlyIncome|ListingCategory|EmpIndex|            features|label|prediction|
+----------+------------+----------------+-----------+-------------------+---------------+--------+--------------------+-----+----------+
|         0|         3.0|   Self-employed|      680.0|             6350.0|       Business|     2.0|[3.0,2.0,680.0,63...|    0|       0.0|
|         0|         4.0|        Employed|      640.0|             6250.0|           Debt|     1.0|[4.0,1.0,640.0,62...|    0|       0.0|
|         0|         6.0|        Employed|      640.0|             6000.0|           Debt|     1.0|[6.0,1.0,640.0,60...|    0|       0.0|
|         0|         6.0|        Employed|      660.0|        3928.083333|       Business|     1.0|[6.0,1.0,660.0,39...|    0|       1.0|
|         0|         6.0|       Fu

## Step 7: Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [19]:
predictions_test = rf_model.transform(test)  # Hint : test
predictions_train = rf_model.transform(training)  # Hint : training

### 7.1 Accuracy

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

print("Training set accuracy = " , evaluator.evaluate(predictions_train))
print("Test set accuracy = " , evaluator.evaluate(predictions_test))

Training set accuracy =  0.8163265306122449
Test set accuracy =  0.7358490566037735


#### Is RF more stable than DTs ?
Do a few runs, and see the accuracy above.   
Does it vary a lot like Decision Trees before?   
Probably not.  Why do you think it is?

### 7.2 Confusion Matrix

In [21]:
cm = predictions_test.groupBy('LoanStatus').pivot('prediction', [0,1]).count().na.fill(0).orderBy('LoanStatus')
cm.show()

+----------+---+---+
|LoanStatus|  0|  1|
+----------+---+---+
|         0| 10|  6|
|         1|  8| 29|
+----------+---+---+



In [22]:
import seaborn as sns

cm_pd = cm.toPandas()
cm_pd.set_index("LoanStatus", inplace=True)
# print(cm_pd)

# colormaps : cmap="YlGnBu" , cmap="Greens", cmap="Blues",  cmap="Reds"
sns.heatmap(cm_pd, annot=True, fmt=',', cmap="Blues")

### 7.3 - AUC

For scewed data 'Area Under Precision Recall' curve might be better indicator.  
https://stats.stackexchange.com/questions/90779/area-under-the-roc-curve-or-area-under-the-pr-curve-for-imbalanced-data

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# default metrics for BinaryClassificationEvaluator is 'areaUnderCurve'
evaluator1 = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName='areaUnderROC')

print("AUC for training: " , evaluator1.evaluate(predictions_train))
print ("AUC for test : " , evaluator1.evaluate(predictions_test))

evaluator2 = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName='areaUnderPR')
print("Area under PR for training: " , evaluator2.evaluate(predictions_train))
print ("Area under PR for test : " , evaluator2.evaluate(predictions_test))


AUC for training:  0.9186658249158248
AUC for test :  0.7635135135135137
Area under PR for training:  0.9628402088841111
Area under PR for test :  0.8881679267806732


## Step 8: Improve Accuracy

### Add more data
In Step-1 change the 'datafile' to the full dataset.  
And see how the accuracy above changes

### Add more features
Look at the schema of the full dataset.  Are there any columns you want to add